[Документация](https://docs.python.org/3/library/asyncio.html#module-asyncio)

In [1]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

## Заметки

- **coroutine** - функция с async, которая создаёт coroutine object и исполняется только при:
    - await
    - когда её оборачивают в Task
- **Task** - обёртка над корутиной для планирования её выполнения циклом событий
- **Event loop** - планирует задачи, обрабатывает I/O, колбэки и переключается при await

**asyncio.run(coro)**: создать цикл, выполнить корутину, закрыть цикл.\
**asyncio.create_task(coro)**: запланировать корутину как задачу, вернуть Task.\
**await asyncio.sleep(n)**: Отпускает поток ("Уступает" циклу событий).\
**await gather()**: запускает await сразу для нескольких корутин.

- Пока await из корутины не разрешился, выполнение кода продолжается в цикле событий.
- При завершении корутины, на место await подставляется результат.

## coro vs Task

In [16]:
async def async_coro():
    return None

c = async_coro

res = await c()

print(res)

None


In [12]:
task = asyncio.create_task(async_coro())

await asyncio.sleep(0)
print(task.result())

None


## sync vs async (IO-Bound)

In [18]:
import time

### sync

In [ ]:
def get_data(sleep):
    time.sleep(sleep)
    return f'{sleep} s.'

def get_sync_data():
    start_time = time.time()

    data = [get_data(sleep=sleep) for sleep in range(1, 4)]

    end_time = time.time()
    print(f'Время выполнения (sync): {end_time - start_time:.2f} s')

    return data

get_sync_data()

Время выполнения (sync): 6.00 s


['1 seconds', '2 seconds', '3 seconds']

### async

In [32]:
async def aget_data(sleep):
    await asyncio.sleep(sleep)
    return f'{sleep} s.'

async def get_async_data():
    start_time = time.time()

    funcs = [aget_data(sleep=sleep) for sleep in range(1, 4)]

    data = await asyncio.gather(*funcs)

    end_time = time.time()
    print(f'Время выполнения (async): {end_time - start_time:.2f} s')

    return data

await get_async_data()

Время выполнения (async): 3.00 s


['1 s.', '2 s.', '3 s.']

## Отмена задач

In [44]:
async def some_task(sleep=1):
    while 1:
        await asyncio.sleep(sleep)
        print('task is running...')
        
task0 = asyncio.create_task(some_task())

await asyncio.sleep(3)

task0.cancel()

try:
    await task0
except asyncio.CancelledError:
    print('task is cancelled')

task is running...
task is running...
task is cancelled


## TaskGroup

- Объект задачи содержит внутри себя корутину (`Task(coro=coro, ...)`).
- Задачи внутри TaskGroup запускаются либо при выходе из async with, либо при первом await.

In [50]:
start_time = time.time()

async with asyncio.TaskGroup() as tg: 
    print('Запуск TaskGroup()...')
    
    task1 = tg.create_task(aget_data(1))
    print(f'📥 Добавлена задача: {task1}')

    task2 = tg.create_task(aget_data(2))
    print(f'📥 Добавлена задача: {task2}')

print('✅ TaskGroup() выполнена')
end_time = time.time()
print(f'Время выполнения (async): {end_time - start_time:.2f} s')
    
print(task1.result())
print(task2.result())


Запуск TaskGroup()...
📥 Добавлена задача: <Task pending name='Task-64' coro=<aget_data() running at /tmp/ipykernel_1256118/2943665540.py:1> cb=[TaskGroup._on_task_done()]>
📥 Добавлена задача: <Task pending name='Task-65' coro=<aget_data() running at /tmp/ipykernel_1256118/2943665540.py:1> cb=[TaskGroup._on_task_done()]>
✅ TaskGroup() выполнена
Время выполнения (async): 2.00 s
1 s.
2 s.


## timeout

In [51]:
async def foo(): 
    print('🚀 Запуск foo()...')
    await asyncio.sleep(2)
    print('✅ foo() вернула значение')
    return 'foo'

async def bar(): 
    print('🚀 Запуск bar()...')
    await asyncio.sleep(1)
    print('✅ bar() вернула значение')
    return 'bar'

async def get_something(): 
    
    print('⚙️ Запуск get_something()...')
    
    async with asyncio.timeout(2): 
        foo_result, bar_result = await asyncio.gather(foo(), bar())

    print(foo_result)
    print(bar_result)

try:
    await get_something()
except TimeoutError: 
    print('TimeoutError')

⚙️ Запуск get_something()...
🚀 Запуск foo()...
🚀 Запуск bar()...
✅ bar() вернула значение
TimeoutError


## CPU-bound задачи внутри async
CPU-bound задачи (или синхронный sleep) внутри корутин блокируют поток и не дают идти циклу событий.

In [52]:
async def func1(): 
    print('🚀 Запуск func1()...')
    for i in range(10_000):   # ~ 2s 
        
        # Переключаем контекст для того, что бы дать пройти циклу событий
        if i % 100 == 0:
            await asyncio.sleep(0)
        # Иначе поток заблокируется
        i ** i

    print('✅ func1(): OK')   
    return 'func1'

async def func2(): 
    print('🚀 Запуск func2()...')
    await asyncio.sleep(1)
    print('✅ func2(): OK')   
    return 'func2'

async def run_f1_f2(): 
    start_time = time.time()

    async with asyncio.timeout(2): 
        f1, f2 = await asyncio.gather(func1(), func2())
    
    end_time = time.time()

    print(f'Время выполнения (async): {end_time - start_time:.2f} s')

try:
    asyncio.run(run_f1_f2())
except TimeoutError: 
    print('TimeoutError')

🚀 Запуск func1()...
🚀 Запуск func2()...
✅ func2(): OK
TimeoutError
